In [1]:
def preprocess_edge_list(filename):
    edge_list = []
    with open(filename) as f_handle:
        for line in f_handle:
            edge_list.append(line.split())  # vertices exist as str
            
    #print(edge_list)
    return adj_list(edge_list)


# input: edge list
# output: dict graph
def adj_list(edge_list):

    num_v = int(edge_list[-1][0])  # the tails of edges follow **numerical order**
    vertices = [i for i in range(1, num_v + 1)]
    graph_obj = dict(list((i, []) for i in vertices))    # sink vertex is indicated by the value []
    
    for i, j in edge_list:  # i, j are str
        graph_obj[int(i)].append(int(j))
    
    #print(graph_obj)
    return (graph_obj)


# define vertex, Graph classes
# Vertex class for directed graphs (object with 'key', 'tail_of', and 'head_of' keys)

# remark: In particular, a directed edge is specified as an ordered pair of vertices u, v
# and is denoted by (u, v) or u -> v. In this case, u is the tail of the edge and v is the
# head
class Vertex():
    def __init__(self, key):
        self._key = key
        self._tail_of = {} # where to go
        self._head_of = {} # how to come

    def __str__(self):
        return '{' + "'key': {}, 'tail_of': {}, 'head_of': {}".format(
            self._key,
            self._tail_of,
            self._head_of
        ) + '}'

    def add_head(self, head_key, weight=1):
        if (head_key):
            self._tail_of[head_key] = weight

    def add_tail(self, tail_key, weight=1):
        if (tail_key):
            self._head_of[tail_key] = weight

    def tail_of(self, head_key):
        return head_key in self._tail_of

    def head_of(self, tail_key):
        return tail_key in self._head_of

    def get_tail_of_keys(self):
        return list(self._tail_of.keys())

    def get_head_of_keys(self):
        return list(self._head_of.keys())

    def remove_tail(self, tail_key):
        if tail_key in self._head_of:
            del self._head_of[tail_key]

    def remove_head(self, head_key):
        if head_key in self._tail_of:
            del self._tail_of[head_key]

    def get_tail_weight(self, tail_key):
        if tail_key in self._head_of:
            return self._head_of[tail_key]

    def get_head_weight(self, head_key):
        if head_key in self._tail_of:
            return self._tail_of[head_key]


# Directed graph class
class Graph():
    def __init__(self):
        self._vertices = {}

    # 'x in graph' will use this containment logic
    def __contains__(self, key):
        return key in self._vertices

    # 'for x in graph' will use this iter() definition, where x is a vertex in an array
    def __iter__(self):
        return iter(self._vertices.values())

    def __str__(self):
        output = '\n{\n'
        
        vertices = self._vertices.values()
        for v in vertices:
            graph_key = "{}".format(v._key)
            v_str = "\n   'key': {}, \n   'tail_of': {}, \n   'head_of': {}".format(
                v._key,
                v._tail_of,
                v._head_of
            )
            output += ' ' + graph_key + ': {' + v_str + '\n },\n'
        return output + '}'

    def add_v(self, v):
        if v:
            self._vertices[v._key] = v
        return self

    def get_v(self, key):
        try:
            return self._vertices[key]
        except KeyError:
            return None

    def get_v_keys(self):
        return list(self._vertices.keys())

    # removes vertex as head and tail from all its neighbors, then deletes vertex
    def remove_v(self, key):
        if key in self._vertices:
            head_of_keys = self._vertices[key].get_head_of_keys()
            tail_of_keys = self._vertices[key].get_tail_of_keys()
            for tail_key in head_of_keys:
                self.remove_head(tail_key, key)
            for head_key in tail_of_keys:
                self.remove_tail(key, head_key)
            del self._vertices[key]

    def add_e(self, tail_key, head_key, weight=1):
        if tail_key not in self._vertices:
            self.add_v(Vertex(tail_key))
        if head_key not in self._vertices:
            self.add_v(Vertex(head_key))

        self._vertices[tail_key].add_head(head_key, weight)

    def get_e(self, tail_key, head_key):
        if tail_key and head_key in self._vertices:
            if ((head_key in G.get_v(tail_key).get_tail_of_keys()) & 
                (tail_key in G.get_v(head_key).get_head_of_keys())):
                    return self.get_v(tail_key).get_head_weight(head_key)
        return ("NULL")
            

    # adds the weight for an edge if it exists already, with a default of 1
    def increase_e(self, tail_key, head_key, weight=1):
        if tail_key not in self._vertices:
            self.add_v(Vertex(tail_key))
        if head_key not in self._vertices:
            self.add_v(Vertex(head_key))

        w_v1_v2 = self.get_v(tail_key).get_head_weight(head_key)
        new_w_v1_v2 = w_v1_v2 + weight if w_v1_v2 else weight

        self._vertices[tail_key].add_head(head_key, new_w_v1_v2)
        
    def update_e(self, tail_key, head_key, weight=1):
        if tail_key not in self._vertices:
            self.add_v(Vertex(tail_key))
        if head_key not in self._vertices:
            self.add_v(Vertex(head_key))

        new_w_v1_v2 = weight if weight else 1

        self._vertices[tail_key].add_head(head_key, new_w_v1_v2)

    def has_forward_e(self, tail_key, head_key):
        if tail_key in self._vertices:
            return self._vertices[tail_key].tail_of(head_key)

    def remove_tail(self, tail_key, head_key):
        if head_key in self._vertices:
            self._vertices[head_key].remove_tail(tail_key)

    def remove_head(self, tail_key, head_key):
        if tail_key in self._vertices:
            self._vertices[tail_key].remove_head(head_key)

    def remove_e(self, tail_key, head_key):
        if tail_key in self._vertices:
            self._vertices[tail_key].remove_head(head_key)
        if head_key in self._vertices:
            self._vertices[head_key].remove_tail(tail_key)

    def for_each_v(self, cb):
        for v in self._vertices:
            cb(v)
            
            
# input: adj_list(graph_object)
# output: Graph
def creat_graph(graph_obj):
    G = Graph()
    for v_key in graph_obj:  # equavilant of for v_key in vertices
        
        # establish vertex v for adding heads and tails
        # the 'else' condition is to capture already processed vertices
        v = Vertex(v_key) if v_key not in G else G.get_v(v_key)
        
        # v_key -> head_key
        for head_key in graph_obj[v_key]:
            
            # forward
            v.add_head(head_key)
            
            # reversed
            v_head = Vertex(head_key) if head_key not in G else G.get_v(head_key)
            v_head.add_tail(v_key)
            
            # save changes
            G.add_v(v_head)
            
        G.add_v(v)
        
    return(G)

In [5]:
import numpy as np
# input: file of adjlist with weight (file name)
# output: dictionary of adjlist with weight (graph_obj)
def preprocess_adj_list_with_weights(filename):
    adj_list_with_weights = {}
    with open(filename) as f_handle:
        for line in f_handle:
            line = line.split()
            v = int(line[0])
            line = np.array([i.split(',') for i in line[1:]]).astype('int').tolist() #[int(a), int(b) for i in line[1:] for (a, b) in i.split(',')]
            #line = np.array(line).astype('int')
            adj_list_with_weights[v] = line
            
    return(adj_list_with_weights)

# input: graph_obj
# output: graph
def creat_undirected_graph_with_weights(graph_obj):
    G = Graph()
    for v_key in graph_obj:
        v = Vertex(v_key) if v_key not in G else G.get_v(v_key)
        for head_key, w in graph_obj[v_key]:
            v.add_head(head_key, w)
            v.add_tail(head_key, w)
            v_head = Vertex(head_key) if head_key not in G else G.get_v(head_key)
            v_head.add_tail(v_key, w)
            v_head.add_head(v_key, w)
            G.add_v(v_head)
        
        G.add_v(v)
        
    return(G)

In [3]:
from collections import deque as Stack
import numpy as np
# input: G = (V, E) with nonnegative weight and no subgraph for each edge
#        and a starting node s
# output: shortest (s, v) for every v in V
# Dijkstra's Algorithm is based on the BFS
# This one is not entirely correct
def dijkstra(G, s):
    # stack for traversing
    X = Stack([s])
    
    # vertex set
    V = G.get_v_keys()
    
    # storing nodes in already explored layer
    # not currently being explored layer
    E = dict(zip(V, [False]*len(V)))
    #E[s] = True
    
    # path
    P = dict(zip(V, [[]]*len(V)))
    P[s] = [s]
        
    # length
    L = dict(zip(V, [np.infty]*len(V)))
    L[s] = 0
    
    while X:
        # real explored
        # BFS is executed on the node s
        s = X.pop()
        E[s] = True

        # greedy probing, BFS not yet executed on node v, 
        # not considered explored
        # v are heads of s
        for v in G.get_v(s).get_tail_of_keys():
            # appendleft+pop guarantees BFS from DFS (append+pop)
            if (not E[v]):
                X.appendleft(v)
                #E[v] = True
        
            # s is tail of v
            l = G.get_e(s, v)
            # greedy condition
            if L[s] + l < L[v]:
                # total length
                L[v] = L[s] + l
                # complete path
                P[v] = P[s] + [v]
    return L, P


In [9]:
from collections import deque as Stack
import numpy as np
# input: G = (V, E) with nonnegative weight and no subgraph for each edge
#        and a starting node s
# output: shortest (s, v) for every v in V
def dijkstra(G, s):
    # vertex set
    V = G.get_v_keys()
    # for traversing
    X = V
    
    # storing nodes in already explored layer
    # not currently being explored layer
    E = dict(zip(V, [False]*len(V)))
    #E[s] = True
    
    # path
    P = dict(zip(V, [[]]*len(V)))
    P[s] = [s]
        
    # length
    L = dict(zip(V, [np.infty]*len(V)))
    L[s] = 0
    
    while X:
        # real explored
        # BFS is executed on the node s
        min = np.infty
        # each time select the node with the 
        # minimum length from the initial node
        # greedy condition 1
        for v in X:
            if L[v] < min:
                min = L[v]
                u = v
        # this can be achieved with the more efficient
        # enqueue(dequeue) but I'm lazy
        X.remove(u)
        # This node is optimized
        E[u] = True

        # greedy probing, BFS not yet executed on node v, 
        # not considered explored
        # v are heads of s
        for v in G.get_v(u).get_tail_of_keys():
            # we don't need to look back on path
            if (E[v]):
                continue
            
            # s is tail of v
            l = G.get_e(u, v)
            # greedy condition 2
            if L[u] + l < L[v]:
                # total length
                L[v] = L[u] + l
                # complete path
                P[v] = P[u] + [v]
    return L, P


// Initialization  
1 X := {s}  
2 len(s) := 0, len(v) := +infty for every v != s  
// Main loop  
3 while there is an edge (v, w) with v in X, w not in X do  
    4 (v*, w*) := such an edge minimizing len(v) + lvw  
    5 add w* to X  
    6 len(w*) := len(v*) + `v*w  


 1  function Dijkstra(Graph, source):  
 2
 3      for each vertex v in Graph.Vertices:            
 4          dist[v] ← INFINITY                 
 5          prev[v] ← UNDEFINED                
 6          add v to Q                     
 7      dist[source] ← 0                       
 8     
 9      while Q is not empty:  
10          u ← vertex in Q with min dist[u]   
11          remove u from Q  
12                                        
13          for each neighbor v of u still in Q:  
14              alt ← dist[u] + Graph.Edges(u, v)  
15              if alt < dist[v]:              
16                  dist[v] ← alt  
17                  prev[v] ← u  
18
19      return dist[], prev[]  

In [12]:
graph_obj = preprocess_adj_list_with_weights('dijkstraData.txt')
G = creat_undirected_graph_with_weights(graph_obj)

In [13]:
t = []
v = [7,37,59,82,99,115,133,165,188,197]
L, P = dijkstra(G, 1)
for i in v:
   t.append(L[i])

In [14]:
t # wrong??? Ahhh now good

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]

Correct ans:  

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, **2505**, 3068]

#### Debug section for dijkstra v1

In [149]:
P[188]

[1, 114, 103, 196, 188]

In [147]:
sum = 0
for i in range(len(P[188])-1):
    print(G.get_e(P[188][i], P[188][i+1]))
    sum += G.get_e(P[188][i], P[188][i+1])
    
sum

508
1318
696
88


2610